In [1]:
import requests
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import json
import os
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import platform

In [2]:
from Webtoon import Webtoon 
from Episode import Episode 
from Platform import Platform 
from Crawler import Crawler 

In [3]:
ChromeDriverManager().install()

'C:\\Users\\pyj78\\.wdm\\drivers\\chromedriver\\win64\\119.0.6045.105\\chromedriver-win32/chromedriver.exe'

In [4]:
driver = webdriver.Chrome()

In [5]:
days = [
        "mon",
        "tue",
        "wed", 
        "thu", 
        "fri",
        "sat",
        "sun"
    ]
id = 0

In [6]:
unreachableUrls = []
id = 0

In [7]:
class KakaoPageCrawler(Crawler):
    def __init__(self, driver, day):
        self.driver = driver
        self.platformName = "kakaoPage"
        self.day = day

        self.platform = Platform()
        self.platform.setName(self.platformName)
        self.platform.setDay(self.day)
        
        self.createFolder()
        
    def crawl(self):
        
        dayToNumberTable = {
            "mon": 1,
            "tue": 2,
            "wed": 3, 
            "thu": 4, 
            "fri": 5,
            "sat": 6,
            "sun": 7
        }
        
    
        mainPageUrl = "https://page.kakao.com/menu/10010/screen/52?tab_uid=" + str(dayToNumberTable[self.day])
        driver.get(mainPageUrl)
        self.scrolDown()


        webtoonElementList = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div/div[4]/div/div/div/div/a')
        webtoonUrlList = []
        for webtoonElement in webtoonElementList:
            wait = WebDriverWait(webtoonElement, 5)
            webtoonUrl = webtoonElement.get_attribute("href")
            webtoonUrlList.append(webtoonUrl)


        for webtoonUrl in webtoonUrlList:
            try:
                webtoon = self.getWebtoon(webtoonUrl)
                self.platform.addWebtoon(webtoon)
            except:
                unreachableUrls.append(webtoonUrl)
                print(webtoonUrl + " is unreachable")
            global id
            id += 1
            
            
        
        webtoonFilePath = os.path.join(self.platformName, self.platformName + "_" + self.day)
        self.saveToJson(webtoonFilePath, self.platform)
        
    def getWebtoon(self, webtoonUrl):
        self.driver.get(webtoonUrl)
        wait = WebDriverWait(self.driver, 5)
        
        title = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[1]'))).text
        author = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[2]'))).text
        serializedDay = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[2]/span'))).text
        imageUrl = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[1]/div[2]/div[2]/div/div/img'))).get_attribute("src")
        genre = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[1]/div/span[2]'))).text
        
        
        episodes = self.getEpisodes()
        global id
        episodeFilePath = os.path.join( self.platformName, "webtoons","kp" + str(id))
        self.saveToJson(episodeFilePath, episodes)

        summary = self.getSummary(wait)
        webtoon = Webtoon()
        webtoon.setWebtoon(self.platformName, self.day, id, title, author, serializedDay, webtoonUrl, imageUrl, genre, summary )

        

        return webtoon


    def getSummary(self, waitElement):
        infoUrl = waitElement.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[1]/div/div/div[2]/a'))).get_attribute("href")
        self.driver.get(infoUrl)
        wait = WebDriverWait(self.driver, 5)
        
        return wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div/div[1]/div[2]/div[2]/div/div[1]/span'))).text

    
    def getEpisodes(self):
        self.showAllEpisodes()
        episodeList = []
        episodeElementList = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/ul/li')
        for episodeElement in episodeElementList:
            wait = WebDriverWait(episodeElement, 5)
            
            
            title = wait.until(EC.presence_of_element_located((By.XPATH, 'div/div/a/div/div[2]/div[1]'))).text.replace("\n", "")
            url = wait.until(EC.presence_of_element_located((By.XPATH, 'div/div/a'))).get_attribute("href")
            imageUrl = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'img'))).get_attribute("src")
            publicationDate = wait.until(EC.presence_of_element_located((By.XPATH, 'div/div/a/div/div[2]/div[2]/span'))).text
            

            episode = Episode(title, url, imageUrl, publicationDate)

            episodeList.append(episode)

        return episodeList
            
            
    
    def showAllEpisodes(self):
        wait = WebDriverWait(self.driver, 1)
        while True:
            try:
                time.sleep(1)
                moreButton = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/div[2]/img')))
                moreButton.click()
            except Exception as e:
                break
            
    def scrolDown(self):
        while True:
            # 현재 높이 저장
            last_height = self.driver.execute_script("return document.body.scrollHeight")
        
            # 페이지 가장 아래로 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
            # 스크롤이 끝나기까지 잠시 대기
            time.sleep(1)
        
            # 새로운 높이 계산
            new_height = self.driver.execute_script("return document.body.scrollHeight")
        
            # 더 이상 스크롤할 내용이 없으면 반복 종료
            if new_height == last_height:
                break

        # body = driver.find_element(By.CSS_SELECTOR, 'body')
        # body.send_keys(Keys.END)
        


In [8]:
# for day in days:
#     kakaoPageCrawler = KakaoPageCrawler(driver, day)
#     kakaoPageCrawler.crawl()

In [9]:
kakaoPageCrawler = KakaoPageCrawler(driver, "mon")
kakaoPageCrawler.crawl()

Folder 'C:\Users\pyj78\Desktop\webtoon\kakaoPage' created successfully
Folder 'kakaoPage\webtoons' created successfully
https://page.kakao.com/content/62743785 is unreachable
https://page.kakao.com/content/62851300 is unreachable
https://page.kakao.com/content/62101502 is unreachable
https://page.kakao.com/content/62407115 is unreachable
https://page.kakao.com/content/61271601 is unreachable
https://page.kakao.com/content/62473764 is unreachable
https://page.kakao.com/content/61453042 is unreachable
https://page.kakao.com/content/56702100 is unreachable
https://page.kakao.com/content/54884637 is unreachable
https://page.kakao.com/content/60561657 is unreachable
https://page.kakao.com/content/61770596 is unreachable
https://page.kakao.com/content/60257175 is unreachable
https://page.kakao.com/content/61547597 is unreachable
https://page.kakao.com/content/59942846 is unreachable
https://page.kakao.com/content/55482233 is unreachable
https://page.kakao.com/content/55566760 is unreachable


In [10]:
kakaoPageCrawler = KakaoPageCrawler(driver, "tue")
kakaoPageCrawler.crawl()

Folder 'C:\Users\pyj78\Desktop\webtoon\kakaoPage' already exists
Folder 'kakaoPage\webtoons' already exists
https://page.kakao.com/content/63046544 is unreachable
https://page.kakao.com/content/63024185 is unreachable
https://page.kakao.com/content/60482121 is unreachable
https://page.kakao.com/content/62011604 is unreachable
https://page.kakao.com/content/60531893 is unreachable
https://page.kakao.com/content/62777301 is unreachable


In [11]:
kakaoPageCrawler = KakaoPageCrawler(driver, "wed")
kakaoPageCrawler.crawl()

Folder 'C:\Users\pyj78\Desktop\webtoon\kakaoPage' already exists
Folder 'kakaoPage\webtoons' already exists
https://page.kakao.com/content/54097394 is unreachable
https://page.kakao.com/content/58743054 is unreachable
https://page.kakao.com/content/62961293 is unreachable


In [ ]:
kakaoPageCrawler = KakaoPageCrawler(driver, "thu")
kakaoPageCrawler.crawl()

In [ ]:
kakaoPageCrawler = KakaoPageCrawler(driver, "fri")
kakaoPageCrawler.crawl()

In [ ]:
kakaoPageCrawler = KakaoPageCrawler(driver, "sat")
kakaoPageCrawler.crawl()

In [ ]:
kakaoPageCrawler = KakaoPageCrawler(driver, "sun")
kakaoPageCrawler.crawl()

In [12]:
len(unreachableUrls)

66

In [13]:
unreachableUrls

['https://page.kakao.com/content/62743785',
 'https://page.kakao.com/content/62851300',
 'https://page.kakao.com/content/62101502',
 'https://page.kakao.com/content/62407115',
 'https://page.kakao.com/content/61271601',
 'https://page.kakao.com/content/62473764',
 'https://page.kakao.com/content/61453042',
 'https://page.kakao.com/content/56702100',
 'https://page.kakao.com/content/54884637',
 'https://page.kakao.com/content/60561657',
 'https://page.kakao.com/content/61770596',
 'https://page.kakao.com/content/60257175',
 'https://page.kakao.com/content/61547597',
 'https://page.kakao.com/content/59942846',
 'https://page.kakao.com/content/55482233',
 'https://page.kakao.com/content/55566760',
 'https://page.kakao.com/content/62676232',
 'https://page.kakao.com/content/61192172',
 'https://page.kakao.com/content/57611444',
 'https://page.kakao.com/content/61871204',
 'https://page.kakao.com/content/48511988',
 'https://page.kakao.com/content/62313653',
 'https://page.kakao.com/content